In [1]:
cd /scratch/aqd215/k-gnn/examples/

/scratch/aqd215/k-gnn/examples


In [2]:
import os.path as osp

import argparse
import torch
import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU
from torch_scatter import scatter_mean
from torch_geometric.datasets import QM9
import torch_geometric.transforms as T
from torch_geometric.nn import NNConv
from k_gnn import GraphConv, DataLoader, avg_pool
from k_gnn import ConnectedThreeMalkin
import sys

In [3]:
class MyFilter(object):
    def __call__(self, data):
        return data.num_nodes > 6  # Remove graphs with less than 6 nodes.


class MyTransform(object):
    def __call__(self, data):
        data.y = data.y[:, 0]  # Specify target: 0 = mu
        return data

In [4]:
target = 0
path = osp.join(osp.dirname(osp.realpath("__file__")), '..', 'data', '1-QM9')
dataset = QM9(path, transform=T.Compose([MyTransform()]))

In [5]:
dataset.data

Data(edge_attr=[4823498, 4], edge_index=[2, 4823498], idx=[129433], name=[129433], pos=[2333625, 3], x=[2333625, 13], y=[129433, 19], z=[2333625])

In [12]:
test_loader = DataLoader(dataset[2:3], batch_size=64)
val_loader = DataLoader(dataset[1:2], batch_size=64)
train_loader = DataLoader(dataset[:1], batch_size=64)

In [13]:
for data in test_loader:
    print(data, '\n', data.x, '\n', data.edge_attr, '\n', data.edge_index, '\n', data.y)

Batch(batch=[3], edge_attr=[4, 4], edge_index=[2, 4], idx=[1], name=[1], pos=[3, 3], x=[3, 13], y=[1], z=[3]) 
 tensor([[0., 0., 0., 1., 0., 8., 0., 0., 0., 0., 0., 1., 2.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]]) 
 tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.]]) 
 tensor([[0, 0, 1, 2],
        [1, 2, 0, 0]]) 
 tensor([1.8511])


In [7]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        M_in, M_out = 37,64 #data.num_features, 32 # 
        nn1 = Sequential(Linear(4, 128), ReLU(), Linear(128, M_in * M_out))
        self.conv1 = NNConv(M_in, M_out, nn1)

        M_in, M_out = M_out, 64
        nn2 = Sequential(Linear(4, 128), ReLU(), Linear(128, M_in * M_out))
        self.conv2 = NNConv(M_in, M_out, nn2)

        M_in, M_out = M_out, 64
        nn3 = Sequential(Linear(4, 128), ReLU(), Linear(128, M_in * M_out))
        self.conv3 = NNConv(M_in, M_out, nn3)

        self.fc1 = torch.nn.Linear(64, 32)
        self.fc2 = torch.nn.Linear(32, 16)
        self.fc3 = torch.nn.Linear(16, 1)
        
        self.initialize_weights()

    def forward(self, data):
        x = data.x
        x = F.elu(self.conv1(x, data.edge_index, data.edge_attr))
        x = F.elu(self.conv2(x, data.edge_index, data.edge_attr))
        x = F.elu(self.conv3(x, data.edge_index, data.edge_attr))

#         x = scatter_mean(x, data.batch, dim=0)

        x = F.elu(self.fc1(x))
        x = F.elu(self.fc2(x))
        x = self.fc3(x)
        return x.view(-1)
    
    def initialize_weights(self):
        for m in self.modules():
#             print(m)
            if isinstance(m, Sequential):
                for elem in m:
                    if isinstance(elem, Linear):
                        torch.nn.init.kaiming_uniform_(elem.weight)
            elif isinstance(m, Linear):
                torch.nn.init.kaiming_uniform_(elem.weight)


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, factor=0.7, patience=5, min_lr=0.00001)

In [9]:
def train(epoch):
    model.train()
    loss_all = 0

    for data in train_loader:
        data.x = torch.tensor([[7., 0., 0., 1., 0., 0., 0., 0., 0., 4., 0., 0., 0., 1., 0., 0., 0., 0.,0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.,0.],[6., 0., 1., 0., 0., 0., 0., 0., 0., 4., 0., 0., 0., 1., 0., 0., 0., 0.,0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.,0.],[6., 0., 1., 0., 0., 0., 0., 0., 0., 4., 0., 0., 0., 1., 0., 0., 0., 0.,0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.,0.],[7., 0., 0., 1., 0., 0., 0., 0., 0., 3., 0., 0., 1., 0., 0., 0., 0., 0.,0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.,0.],[6., 0., 1., 0., 0., 0., 0., 0., 0., 4., 0., 0., 0., 1., 0., 0., 0., 0.,0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.,0.],[6., 0., 1., 0., 0., 0., 0., 0., 0., 4., 0., 0., 0., 1., 0., 0., 0., 0.,0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,0.],[6., 0., 1., 0., 0., 0., 0., 0., 0., 4., 0., 0., 0., 1., 0., 0., 0., 0.,0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,0.],[8., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,0.],[6., 0., 1., 0., 0., 0., 0., 0., 0., 4., 0., 0., 0., 1., 0., 0., 0., 0.,0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,0.],[6., 0., 1., 0., 0., 0., 0., 0., 0., 4., 0., 0., 0., 1., 0., 0., 0., 0.,0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,0.],[6., 0., 1., 0., 0., 0., 0., 0., 0., 4., 0., 0., 0., 1., 0., 0., 0., 0.,0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,0.],[1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1.,0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,0.],[1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1.,0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,0.],[1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1.,0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,0.],[1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1.,0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,0.],[1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1.,0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,0.],[1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1.,0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,0.],[1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1.,0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,0.],[1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1.,0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,0.],[1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1.,0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,0.],[1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1.,0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,0.],[1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1.,0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,0.],[1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1.,0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,0.],[1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1.,0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,0.],[1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1.,0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,0.],[1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1.,0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,0.],[1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1.,0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,0.]])
        data.edge_attr = torch.tensor([[0., 1., 0., 0.],[0., 1., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.],[1., 0., 0., 0.]])
        data.edge_index = torch.tensor([[ 0,  1,  4,  5,  4,  6,  1,  2,  0,  7,  2,  3,  2,  8,  3,  4,  2,  9,4,  0,  3, 10,  1, 11,  5, 12,  5, 13,  5, 14,  6, 15,  6, 16,  6, 17,8, 18,  8, 19,  8, 20,  9, 21,  9, 22,  9, 23, 10, 24, 10, 25, 10, 26],[ 1,  0,  5,  4,  6,  4,  2,  1,  7,  0,  3,  2,  8,  2,  4,  3,  9,  2,0,  4, 10,  3, 11,  1, 12,  5, 13,  5, 14,  5, 15,  6, 16,  6, 17,  6,18,  8, 19,  8, 20,  8, 21,  9, 22,  9, 23,  9, 24, 10, 25, 10, 26, 10]])
        data.y = torch.tensor([0.0000,132.1000,60.5000,0.0000,90.0000,23.5000,23.5000,0.0000,23.3000,23.3000,26.1000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000])
        
        data = data.to(device)
        optimizer.zero_grad()
        loss = F.mse_loss(model(data), data.y)
        loss.backward()
        loss_all += loss * data.num_graphs
        optimizer.step()
    return loss_all / len(train_loader.dataset)


def test(loader):
    model.eval()
    error = 0

    for data in loader:
        data = data.to(device)
        error += ((model(data) * std[target].cuda()) -
                  (data.y * std[target].cuda())).abs().sum().item()  # MAE
    return error / len(loader.dataset)

In [10]:
best_val_error = None
for epoch in range(1, 1501):
    lr = scheduler.optimizer.param_groups[0]['lr']
    loss = train(epoch)
    print('Epoch: {:03d}, LR: {:7f}, Loss: {:.7f}'.format(epoch, lr, loss))

Epoch: 001, LR: 0.001000, Loss: 29229.7460938
Epoch: 002, LR: 0.001000, Loss: 865.3693848
Epoch: 003, LR: 0.001000, Loss: 1320.8010254
Epoch: 004, LR: 0.001000, Loss: 801.1968994
Epoch: 005, LR: 0.001000, Loss: 908.9588623
Epoch: 006, LR: 0.001000, Loss: 804.7210083
Epoch: 007, LR: 0.001000, Loss: 621.3265991
Epoch: 008, LR: 0.001000, Loss: 804.3777466
Epoch: 009, LR: 0.001000, Loss: 563.8712769
Epoch: 010, LR: 0.001000, Loss: 551.7028809
Epoch: 011, LR: 0.001000, Loss: 630.4744263
Epoch: 012, LR: 0.001000, Loss: 566.6524048
Epoch: 013, LR: 0.001000, Loss: 506.1747742
Epoch: 014, LR: 0.001000, Loss: 546.0111694
Epoch: 015, LR: 0.001000, Loss: 474.7767029
Epoch: 016, LR: 0.001000, Loss: 473.9577942
Epoch: 017, LR: 0.001000, Loss: 471.2495728
Epoch: 018, LR: 0.001000, Loss: 441.9447021
Epoch: 019, LR: 0.001000, Loss: 453.9951782
Epoch: 020, LR: 0.001000, Loss: 414.0980225
Epoch: 021, LR: 0.001000, Loss: 390.5586243
Epoch: 022, LR: 0.001000, Loss: 396.9231567
Epoch: 023, LR: 0.001000, Los

Epoch: 200, LR: 0.001000, Loss: 0.0700615
Epoch: 201, LR: 0.001000, Loss: 0.0598089
Epoch: 202, LR: 0.001000, Loss: 0.0491872
Epoch: 203, LR: 0.001000, Loss: 0.0533745
Epoch: 204, LR: 0.001000, Loss: 0.0503556
Epoch: 205, LR: 0.001000, Loss: 0.0477580
Epoch: 206, LR: 0.001000, Loss: 0.0583296
Epoch: 207, LR: 0.001000, Loss: 0.0667039
Epoch: 208, LR: 0.001000, Loss: 0.0841680
Epoch: 209, LR: 0.001000, Loss: 0.1321184
Epoch: 210, LR: 0.001000, Loss: 0.2229384
Epoch: 211, LR: 0.001000, Loss: 0.4295563
Epoch: 212, LR: 0.001000, Loss: 1.2323267
Epoch: 213, LR: 0.001000, Loss: 3.1730287
Epoch: 214, LR: 0.001000, Loss: 10.5905066
Epoch: 215, LR: 0.001000, Loss: 28.6948471
Epoch: 216, LR: 0.001000, Loss: 84.5634689
Epoch: 217, LR: 0.001000, Loss: 102.2380295
Epoch: 218, LR: 0.001000, Loss: 56.0150757
Epoch: 219, LR: 0.001000, Loss: 11.0467348
Epoch: 220, LR: 0.001000, Loss: 55.7916183
Epoch: 221, LR: 0.001000, Loss: 39.9825897
Epoch: 222, LR: 0.001000, Loss: 6.7121091
Epoch: 223, LR: 0.001000,

Epoch: 406, LR: 0.001000, Loss: 0.0021066
Epoch: 407, LR: 0.001000, Loss: 0.0020602
Epoch: 408, LR: 0.001000, Loss: 0.0020158
Epoch: 409, LR: 0.001000, Loss: 0.0019704
Epoch: 410, LR: 0.001000, Loss: 0.0019272
Epoch: 411, LR: 0.001000, Loss: 0.0018851
Epoch: 412, LR: 0.001000, Loss: 0.0018431
Epoch: 413, LR: 0.001000, Loss: 0.0018024
Epoch: 414, LR: 0.001000, Loss: 0.0017617
Epoch: 415, LR: 0.001000, Loss: 0.0017228
Epoch: 416, LR: 0.001000, Loss: 0.0016846
Epoch: 417, LR: 0.001000, Loss: 0.0016468
Epoch: 418, LR: 0.001000, Loss: 0.0016101
Epoch: 419, LR: 0.001000, Loss: 0.0015735
Epoch: 420, LR: 0.001000, Loss: 0.0015387
Epoch: 421, LR: 0.001000, Loss: 0.0015041
Epoch: 422, LR: 0.001000, Loss: 0.0014697
Epoch: 423, LR: 0.001000, Loss: 0.0014366
Epoch: 424, LR: 0.001000, Loss: 0.0014043
Epoch: 425, LR: 0.001000, Loss: 0.0013723
Epoch: 426, LR: 0.001000, Loss: 0.0013413
Epoch: 427, LR: 0.001000, Loss: 0.0013102
Epoch: 428, LR: 0.001000, Loss: 0.0012807
Epoch: 429, LR: 0.001000, Loss: 0.

Epoch: 611, LR: 0.001000, Loss: 0.0000106
Epoch: 612, LR: 0.001000, Loss: 0.0000103
Epoch: 613, LR: 0.001000, Loss: 0.0000101
Epoch: 614, LR: 0.001000, Loss: 0.0000097
Epoch: 615, LR: 0.001000, Loss: 0.0000094
Epoch: 616, LR: 0.001000, Loss: 0.0000091
Epoch: 617, LR: 0.001000, Loss: 0.0000089
Epoch: 618, LR: 0.001000, Loss: 0.0000086
Epoch: 619, LR: 0.001000, Loss: 0.0000084
Epoch: 620, LR: 0.001000, Loss: 0.0000081
Epoch: 621, LR: 0.001000, Loss: 0.0000079
Epoch: 622, LR: 0.001000, Loss: 0.0000077
Epoch: 623, LR: 0.001000, Loss: 0.0000075
Epoch: 624, LR: 0.001000, Loss: 0.0000073
Epoch: 625, LR: 0.001000, Loss: 0.0000070
Epoch: 626, LR: 0.001000, Loss: 0.0000068
Epoch: 627, LR: 0.001000, Loss: 0.0000066
Epoch: 628, LR: 0.001000, Loss: 0.0000065
Epoch: 629, LR: 0.001000, Loss: 0.0000063
Epoch: 630, LR: 0.001000, Loss: 0.0000061
Epoch: 631, LR: 0.001000, Loss: 0.0000059
Epoch: 632, LR: 0.001000, Loss: 0.0000057
Epoch: 633, LR: 0.001000, Loss: 0.0000055
Epoch: 634, LR: 0.001000, Loss: 0.

Epoch: 817, LR: 0.001000, Loss: 0.0000000
Epoch: 818, LR: 0.001000, Loss: 0.0000000
Epoch: 819, LR: 0.001000, Loss: 0.0000000
Epoch: 820, LR: 0.001000, Loss: 0.0000000
Epoch: 821, LR: 0.001000, Loss: 0.0000000
Epoch: 822, LR: 0.001000, Loss: 0.0000000
Epoch: 823, LR: 0.001000, Loss: 0.0000000
Epoch: 824, LR: 0.001000, Loss: 0.0000000
Epoch: 825, LR: 0.001000, Loss: 0.0000000
Epoch: 826, LR: 0.001000, Loss: 0.0000000
Epoch: 827, LR: 0.001000, Loss: 0.0000000
Epoch: 828, LR: 0.001000, Loss: 0.0000000
Epoch: 829, LR: 0.001000, Loss: 0.0000000
Epoch: 830, LR: 0.001000, Loss: 0.0000000
Epoch: 831, LR: 0.001000, Loss: 0.0000000
Epoch: 832, LR: 0.001000, Loss: 0.0000000
Epoch: 833, LR: 0.001000, Loss: 0.0000000
Epoch: 834, LR: 0.001000, Loss: 0.0000000
Epoch: 835, LR: 0.001000, Loss: 0.0000000
Epoch: 836, LR: 0.001000, Loss: 0.0000000
Epoch: 837, LR: 0.001000, Loss: 0.0000000
Epoch: 838, LR: 0.001000, Loss: 0.0000000
Epoch: 839, LR: 0.001000, Loss: 0.0000000
Epoch: 840, LR: 0.001000, Loss: 0.

Epoch: 1024, LR: 0.001000, Loss: 0.3734351
Epoch: 1025, LR: 0.001000, Loss: 0.4177614
Epoch: 1026, LR: 0.001000, Loss: 0.2991917
Epoch: 1027, LR: 0.001000, Loss: 0.3649381
Epoch: 1028, LR: 0.001000, Loss: 0.2734947
Epoch: 1029, LR: 0.001000, Loss: 0.2991608
Epoch: 1030, LR: 0.001000, Loss: 0.2212166
Epoch: 1031, LR: 0.001000, Loss: 0.2364379
Epoch: 1032, LR: 0.001000, Loss: 0.1608180
Epoch: 1033, LR: 0.001000, Loss: 0.1848759
Epoch: 1034, LR: 0.001000, Loss: 0.1477481
Epoch: 1035, LR: 0.001000, Loss: 0.1537188
Epoch: 1036, LR: 0.001000, Loss: 0.1322937
Epoch: 1037, LR: 0.001000, Loss: 0.1318352
Epoch: 1038, LR: 0.001000, Loss: 0.1114603
Epoch: 1039, LR: 0.001000, Loss: 0.1112225
Epoch: 1040, LR: 0.001000, Loss: 0.1069954
Epoch: 1041, LR: 0.001000, Loss: 0.0956261
Epoch: 1042, LR: 0.001000, Loss: 0.0981821
Epoch: 1043, LR: 0.001000, Loss: 0.0896367
Epoch: 1044, LR: 0.001000, Loss: 0.0926855
Epoch: 1045, LR: 0.001000, Loss: 0.0861868
Epoch: 1046, LR: 0.001000, Loss: 0.0929396
Epoch: 1047

Epoch: 1228, LR: 0.001000, Loss: 0.0000626
Epoch: 1229, LR: 0.001000, Loss: 0.0000598
Epoch: 1230, LR: 0.001000, Loss: 0.0000572
Epoch: 1231, LR: 0.001000, Loss: 0.0000547
Epoch: 1232, LR: 0.001000, Loss: 0.0000522
Epoch: 1233, LR: 0.001000, Loss: 0.0000499
Epoch: 1234, LR: 0.001000, Loss: 0.0000476
Epoch: 1235, LR: 0.001000, Loss: 0.0000455
Epoch: 1236, LR: 0.001000, Loss: 0.0000434
Epoch: 1237, LR: 0.001000, Loss: 0.0000415
Epoch: 1238, LR: 0.001000, Loss: 0.0000396
Epoch: 1239, LR: 0.001000, Loss: 0.0000378
Epoch: 1240, LR: 0.001000, Loss: 0.0000361
Epoch: 1241, LR: 0.001000, Loss: 0.0000345
Epoch: 1242, LR: 0.001000, Loss: 0.0000329
Epoch: 1243, LR: 0.001000, Loss: 0.0000315
Epoch: 1244, LR: 0.001000, Loss: 0.0000300
Epoch: 1245, LR: 0.001000, Loss: 0.0000287
Epoch: 1246, LR: 0.001000, Loss: 0.0000274
Epoch: 1247, LR: 0.001000, Loss: 0.0000261
Epoch: 1248, LR: 0.001000, Loss: 0.0000250
Epoch: 1249, LR: 0.001000, Loss: 0.0000238
Epoch: 1250, LR: 0.001000, Loss: 0.0000227
Epoch: 1251

Epoch: 1431, LR: 0.001000, Loss: 0.0000000
Epoch: 1432, LR: 0.001000, Loss: 0.0000000
Epoch: 1433, LR: 0.001000, Loss: 0.0000000
Epoch: 1434, LR: 0.001000, Loss: 0.0000000
Epoch: 1435, LR: 0.001000, Loss: 0.0000000
Epoch: 1436, LR: 0.001000, Loss: 0.0000000
Epoch: 1437, LR: 0.001000, Loss: 0.0000000
Epoch: 1438, LR: 0.001000, Loss: 0.0000000
Epoch: 1439, LR: 0.001000, Loss: 0.0000000
Epoch: 1440, LR: 0.001000, Loss: 0.0000000
Epoch: 1441, LR: 0.001000, Loss: 0.0000000
Epoch: 1442, LR: 0.001000, Loss: 0.0000000
Epoch: 1443, LR: 0.001000, Loss: 0.0000000
Epoch: 1444, LR: 0.001000, Loss: 0.0000000
Epoch: 1445, LR: 0.001000, Loss: 0.0000000
Epoch: 1446, LR: 0.001000, Loss: 0.0000000
Epoch: 1447, LR: 0.001000, Loss: 0.0000000
Epoch: 1448, LR: 0.001000, Loss: 0.0000000
Epoch: 1449, LR: 0.001000, Loss: 0.0000000
Epoch: 1450, LR: 0.001000, Loss: 0.0000000
Epoch: 1451, LR: 0.001000, Loss: 0.0000000
Epoch: 1452, LR: 0.001000, Loss: 0.0000000
Epoch: 1453, LR: 0.001000, Loss: 0.0000000
Epoch: 1454